In [37]:
from MMT_reduce_module import *

## MMT_reduce_module Testing

#### The purpose of this notebook is to test functions in the MMT module one-by-one and examine the outputs at each step.

Major to-do items (April 4, 2019): 

* Fix saturated data!
    * Examine how sky frames are being made 
    * Examine how sky subtraction is being performed on science data
    * Determine best practice of aligning saturated frames
    
    
* Determine how flatfield generation is being done and improve if need be
    * Debug negative dark frames/negative flat frame issues?

In [38]:
ls

Calibrations/                     TestingNotebook_MMT_Module.ipynb*
HIP_Test/                         __pycache__/


### Set up paths to raw data and definitions

In [39]:
path_to_raw_sci = './HIP_Test/'
path_to_raw_darks = './Calibrations/darks/dark_13/30.0/'
path_to_raw_flats = './Calibrations/flat/30.0/'
objname = 'HIP106811_TEST_30s'
flattype = 1
saturated = 0
alignflag = 0
imsize = 1024

## The following cells are each of the steps of the reduce_raw_sci function

In [40]:
# Make list of science frames and check exposure time 
scilist = glob.glob(path_to_raw_sci + 'im*.fits')

print(f"Number of science frames found: {len(scilist)} \n")


Number of science frames found: 4 



In [41]:
scitimes = [fits.getheader(im, ignore_missing_end = True)['EXPTIME'] for im in scilist]

# check if all of the exposure times in the current directory are the same:
if all(x == scitimes[0] for x in scitimes):
    print("Science frame exposure time: " + str(scitimes[0]) + "\n")
else:
    raise Exception("Exposure times for given list of files do not match. \
    You may need to make/define separate subfolders for different exptimes.")

sci_exptime = scitimes[0]

n = len(scilist)


# get header from science frames to work with
sciheader = fits.getheader(scilist[0])


Science frame exposure time: 30.0



In [42]:
# check for datacubes
if len(fits.getdata(scilist[0]).shape) == 3: # check for data cubes of science frames
    sciarray = np.zeros([imsize,imsize,n*fits.getdata(scilist[0]).shape[0]])
else:
    sciarray = np.zeros([imsize,imsize,n])

if len(fits.getdata(scilist[0]).shape) == 3: # check for data cubes of science frames    
    totalframes = n*fits.getdata(scilist[0]).shape[0]
else:
    totalframes = n

print(sciarray)

[[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  ...
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  ...
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  ...
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 ...

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  ...
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  ...
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  ...
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]


In [43]:
# in case data were rotated during observing sequence, set up empty array of rotation angles
angle = np.zeros(totalframes)

im_index = 0

for ii in range(0, n):
    
    im = fits.getdata(scilist[ii], ignore_missing_end=True)
    header = fits.getheader(scilist[ii],ignore_missing_end=True)
    
    if len(im.shape) == 3: # check for data cubes of science frames
        assert not np.any(np.isnan(im))
        for jj in range(0, im.shape[0]):
            sciarray[:,:,im_index] = im[jj,:,:]
            angle[im_index] = (header['PA'] - header['ROT']) * (np.pi/180.0)
            im_index += 1
    else: 
        sciarray[:,:,ii] = im  
        angle[ii] = (header['PA'] - header['ROT']) * (np.pi/180.0)
    header = fits.getheader(scilist[ii], ignore_missing_end=True)





In [44]:
print(angle)
print(im)

[1.25918801 1.25795522 1.25668215 1.25527345]
[[   0 1245 1094 ...  929  918  892]
 [   0 1064 1032 ...  910  887  956]
 [   0  889  918 ...  918  973  957]
 ...
 [   0 1042 1071 ... 1069 1157 1514]
 [   0 1023 1074 ... 1075 1303 1915]
 [   0  905 1053 ... 1219 1574 1904]]


In [45]:
#MASTER_DARK

print("Creating and applying master darks and flats...\n")    

# create master dark matching science exposure times
med_dark = dark_combine(path_to_raw_darks, sci_exptime, imsize, objname) 


Creating and applying master darks and flats...

Found 11 darks with exposure times of 30.0. 



In [46]:

# subtract off the median dark frame from each of the science frames
for ii in range (0, totalframes):
    sciarray[:,:,ii] -= med_dark


In [47]:
#MEDIAN_AND_MASTER_FLAT

# create the masterflat 
med_flat, master_flat, flatheader = process_flats(path_to_raw_flats, path_to_raw_darks, imsize, flattype, objname)


Flatfield exposure time: 30.0



In [48]:

# divide each science frame by the masterflat frame
for ii in range(0, totalframes):
    sciarray[:,:,ii] /= master_flat




In [49]:
#BAD_PIXEL_MAP

print("Creating bad pixel map and correcting for bad pixels and cosmic rays. \n",
     "This may take a moment... \n") 

# create bad pixel map
badflat = badpixelmap(med_flat, objname, flatheader)  

Creating bad pixel map and correcting for bad pixels and cosmic rays. 
 This may take a moment... 



In [50]:

# correct the bad pixels and cosmic rays
reduced_sciarray = correct_bad_pixels(sciarray, badflat)


# some housekeeping to free memory later
#del sciarray
print(reduced_sciarray)

Correcting image #0
Correcting image #1
Correcting image #2
Correcting image #3
[[[1323.97135815 1294.71934927 1296.00802068 1308.26019735]
  [1319.27427388 1294.71934927 1300.62678706 1296.02511774]
  [1323.97135815 1301.45343254 1308.70944047 1325.5537155 ]
  ...
  [1333.23233818 1313.96382157 1319.04265741 1316.54756164]
  [1333.23233818 1312.46060823 1319.04265741 1314.92846393]
  [1333.23233818 1313.20882293 1317.84745848 1312.66154631]]

 [[1323.97135815 1296.374624   1296.00802068 1301.99076828]
  [1319.27427388 1294.71934927 1300.62678706 1296.02511774]
  [1323.97135815 1303.86397934 1315.00690546 1312.94662106]
  ...
  [1332.80895462 1313.20882293 1319.76596393 1322.32655218]
  [1332.80895462 1312.46060823 1319.76596393 1322.32655218]
  [1332.80895462 1313.20882293 1319.76596393 1322.32655218]]

 [[1323.97135815 1296.374624   1307.78341014 1301.99076828]
  [1321.04914486 1296.374624   1307.78341014 1301.99076828]
  [1325.19150878 1315.93589362 1323.61645331 1325.5537155 ]
  ..

In [51]:
#MASTER_SKY_A_AND_B

print("Creating master sky from science frames...\n") 

# create median sky from stack of science images
sky_output = create_sky_frames(reduced_sciarray, sciheader, objname, angle)

Creating master sky from science frames...



/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [52]:
# apply sky subtraction to each science image 
skysub_science_array, rot_flag = sky_subtract(reduced_sciarray, sky_output, angle)

/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:3250: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


In [55]:
rot_flag

1

In [56]:
#INDIVIDUAL_REDUCED_SCI_IMAGES

# initialize blank list to hold all of the reduced science image names
scinames_list = []

t0=time.time()
for ii in range(0, totalframes):
    print(f"Saving reduced frame #{ii}")
    sciname = 'reducedsci_00' + str(ii) + '.fits'
    if ii >= 10:
        sciname = 'reducedsci_0' + str(ii) + '.fits'
    if ii >= 100:
        sciname = 'reducedsci_' + str(ii) + '.fits'
    fits.writeto(sciname, skysub_science_array[:,:,ii], sciheader, overwrite = True, output_verify='silentfix')
    scinames_list.append(sciname)
t1=time.time()

Saving reduced frame #0
Saving reduced frame #1
Saving reduced frame #2
Saving reduced frame #3


In [57]:
# get current directory where reduced frames are written
current_dir = os.getcwd()

# measure star positions in all of the images
xcen, ycen = measure_star_centers(skysub_science_array, scinames_list, sciheader, saturated, alignflag, current_dir, saveframes = True)

% Compiled module: MRDFITS.
% Compiled module: FXPOSIT.
% Compiled module: MRD_HREAD.
% Compiled module: FXPAR.
% Compiled module: GETTOK.
% Compiled module: VALID_NUM.
% Compiled module: MRD_SKIP.
% Compiled module: ARRAY_INDICES.
% Compiled module: GCNTRD.
% GCNTRD: Position 20 20 is too far from initial guess
Overwriting existing science frames with star position values: reducedsci_000.fits
% GCNTRD: Position 20 20 is too far from initial guess
Overwriting existing science frames with star position values: reducedsci_001.fits
% GCNTRD: Position 20 20 cannot be fit by a Gaussian
Overwriting existing science frames with star position values: reducedsci_002.fits
Overwriting existing science frames with star position values: reducedsci_003.fits


In [59]:
print('xcen:', xcen, 'ycen:', ycen)

xcen: [698.38623047 716.23120117 707.         721.27734375] ycen: [220.         228.         205.         231.40031433]


In [60]:
#SHIFTED_IMAGES
#STACKED
#FINAL

# final step (!) - shift and combine all of the images.
rotate_shift_align(xcen, ycen, angle, skysub_science_array, objname, sciheader, current_dir, imsize=1024)

t1 = time.time()
timetaken = (t1-t0)/60.
print(f"Completed reduction of {totalframes} images in {timetaken} minutes.")
return

1312 1314
Shifting image 0 of 4...
Shifting image 1 of 4...
Shifting image 2 of 4...
Shifting image 3 of 4...
% Compiled module: MRDFITS.
% Compiled module: FXPOSIT.
% Compiled module: MRD_HREAD.
% Compiled module: FXPAR.
% Compiled module: GETTOK.
% Compiled module: VALID_NUM.
% Compiled module: MRD_SKIP.
% Compiled module: MWRFITS.
% Compiled module: FXADDPAR.
% Compiled module: SXDELPAR.
Completed reduction of 4 images in 3.2855504512786866 minutes.


SyntaxError: 'return' outside function (<ipython-input-60-ee5f5ef0df0c>, line 7)

In [ ]:

reduce_raw_sci(path_to_raw_sci, path_to_raw_darks, path_to_raw_flats, objname, flattype, saturated, alignflag, imsize = 1024)